In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("philosopher0808/real-vs-ai-generated-faces-dataset")

# print("Path to dataset files:", path)

100%|██████████| 7.19G/7.19G [04:05<00:00, 31.5MB/s]

Extracting files...


Path to dataset files: C:\Users\Usuario\.cache\kagglehub\datasets\philosopher0808\real-vs-ai-generated-faces-dataset\versions\1


In [2]:
import os

base_path = r"C:\Users\Usuario\.cache\kagglehub\datasets\philosopher0808\real-vs-ai-generated-faces-dataset\versions\1"

print(os.listdir(base_path))


['dataset', 'data_source']


In [3]:
real_dir = os.path.join(base_path, "real")
fake_dir = os.path.join(base_path, "fake")

print("Real:", len(os.listdir(real_dir)))
print("Fake:", len(os.listdir(fake_dir)))


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\Usuario\\.cache\\kagglehub\\datasets\\philosopher0808\\real-vs-ai-generated-faces-dataset\\versions\\1\\real'